# Long range interaction

Conclusion: implement One-Pion Exchange potential plus contact in $^1S_0$ system and solve Schrödinger equation numerically. Compare solutions with box solutions.

In [ ]:
import numpy as np
from itertools import product
import matplotlib.pylab as plt

In [ ]:
p, wp = np.polynomial.laguerre.laggauss(50)

In [ ]:
HBARC = 197
mN = 939
mu = mN / 2

In [ ]:
H0 = np.diag(p**2 / 2 / mu)

In momentum space, the potential is given by 
$$
    \left\langle \vec p | V | \psi \right\rangle 
    =
    \int \frac{d \vec k^3}{(2\pi)^3} V(\vec p, \vec k) \psi(\vec k)
$$
where in the last case, it was used that the angular momentum is zero ($l=0$).

Choose the Yukawa potential 
$$
    V(\vec p, \vec k) 
    = 
    - \frac{g_A^2}{2 F_\pi^2} 
    \frac{m_\pi^2}{(\vec p - \vec k)^2 + m_\pi^2}
    \leftrightarrow
    V(r) 
    = 
   -\frac{m_{\pi }^2 g_A^2}{8 \pi  F_{\pi }^2} \frac{e^{-m_{\pi }r}}{ r}
$$

$g_A = 1.29$, $f_\pi = 132$MeV and $m_\pi = 141$MeV
The $l = 0$ component is given by



$$
\left\langle\mathbf{k}^{\prime}|\widehat{V}| \mathbf{k}\right\rangle=\frac{2}{\pi} \sum_{l, m} V_{l}\left(k^{\prime}, k\right) Y_{l m}^{*}\left(\Omega_{k^{\prime}}\right) Y_{l m}\left(\Omega_{k}\right)
\qquad
V_{l}\left(k^{\prime}, k\right)=\int_{0}^{\infty} r^{2} d r \, j_{l}(k r) V(r) j_{l}\left(k^{\prime} r\right)
$$
adn thus
$$
 V_0(k, k') = \frac{m_{\pi }^2 g_A^2 }{32 \pi 
   F_{\pi }^2 } \frac{1}{k k'}\log \left(1-\frac{4 k k'}{(k+k')^2+m_{\pi }^2}\right)
$$

In [ ]:
def V(k, mpi=141., gA=1.29, fpi=132.):
    kk = k.reshape(-1, 1)
    return gA**2 * mpi**2/ 32 / np.pi / fpi**2 / kk / k * np.log(1 - 4 * kk * k / ((k+kk)**2 + mpi**2)) 

def V(k):
    return np.ones([k.size]*2)

Using
\begin{align}
    T_{l}\left(k^{\prime}, k ; E\right)
    &=
    V_{l}\left(k^{\prime}, k\right)+\frac{2}{\pi} 
    \int_{0}^{\infty} d q q^{2} 
    \frac{V_{l}\left(k^{\prime}, q\right) T_{l}(q, k ; E)}{E-E_{q}+i \epsilon}
    \\
    &=
    V_{l}\left(k^{\prime}, k\right)
    +\frac{2}{\pi} \mathcal P \int_{0}^{\infty} d q q^{2} 
    \frac{V_{l}\left(k^{\prime}, q\right) T_{l}(q, k ; E)}{E-E_{q}}
    - 2 i \int_{0}^{\infty} d q q^{2} V_{l}\left(k^{\prime}, q\right) T_{l}(q, k ; E) \delta(E-E_{q})
\end{align}
where
$$
    \frac{1}{X+i \epsilon}=\mathcal P \frac{1}{X}-i \pi \delta(X)
    \qquad
    \int d x \,  f(x) \delta(g(x)) = \sum_{x_i} \frac{f(x_i)}{|g'(x_i)|}
$$
was used and $g(x_i) = 0$ are all zeros in the integration domain.
The result is 

$$
    \int_{0}^{\infty} d q q^{2} V_{l}\left(k^{\prime}, q\right) T_{l}(q, k ; E) \delta(E-E_{q})
    =
    \frac{\mu}{q} q^2 \, V_{l}\left(k^{\prime}, q\right) T_{l}(q, k ; E) \bigg|_{q = \sqrt{2 \mu E}}
$$

We can write the on shell $T$ matrix equation as
$$
    \vec t = \vec v + M \vec t \Rightarrow (1 - M) \vec t = \vec v
$$
with 
\begin{align}
    t_i &= T_0(k_i, k_i, E_{k_i})
    \, , &
    v_i &= V_0(k_i, k_i, E_{k_i})
    \, , &
    M_{ij} &=
    \begin{cases}
        \frac{2}{\pi} k_j^2 w_{k_j} \frac{V_0(k_i, k_j)}{E_{k_i} - E_{k_j}} & i \neq j \\
        - 2 i \, \mu k_i \, V_{0}\left(k_i, k_i\right) & i = j
    \end{cases}
\end{align}

In [ ]:
vpp = V(p)
Ep = p**2 / 2 / mu

A = np.zeros([p.size]*2, dtype=complex)
for i, j in product(*[range(p.size)]*2):
    if i == j:
        A[i, j] = 1 + 2j * mu * p[i] * vpp[i, j]
    else:
        A[i, j] = - 2 / np.pi * p[j]**2 * wp[j] * vpp[i, j] / (Ep[i] - Ep[j]) 

In [ ]:
A

In [ ]:
t = np.linalg.solve(A , np.diagonal(vpp))
f = - mu / (2 * np.pi) * t  * 4 * np.pi

In [ ]:
ere = 1 / f + 1j * p

p[np.argmax(ere.real)]

In [ ]:
plt.plot(p, ere.real, ".")

In [ ]:
plt.plot(p, ere.imag, ".")

In [ ]:
plt.plot(p, 1/ (np.linalg.inv(A)@np.diag(vpp)).real, "o")